In [3]:
# import stuff here
import requests
from bs4 import BeautifulSoup

In [1]:
page = requests.get("https://redplanetscience.com/")
page

<Response [200]>

In [4]:
# NASA Mars News
# Scrape the Mars News Site, collect latest News Title and Paragraph Text. 
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
   <title>
    News - Mars Exploration Program
   </title>
  </link>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <s

In [5]:
list(soup.children)

['html',
 '\n',
 <html>
 <head>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
 <link href="css/font.css" rel="stylesheet" type="text/css"/>
 <link href="css/app.css" rel="stylesheet" type="text/css"/>
 <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
 <title>News - Mars Exploration Program</title>
 </link></head>
 <body>
 <div class="col-md-12">
 <div class="row">
 <nav class="navbar navbar-expand-lg navbar-light fixed-top">
 <div class="container-fluid">
 <a class="navbar-brand" href="#">
 <img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
 <sp

In [6]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [7]:
html = list(soup.children)[2]
html

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
<title>News - Mars Exploration Program</title>
</link></head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Progr

In [8]:
list(html.children)

['\n',
 <head>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
 <link href="css/font.css" rel="stylesheet" type="text/css"/>
 <link href="css/app.css" rel="stylesheet" type="text/css"/>
 <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
 <title>News - Mars Exploration Program</title>
 </link></head>,
 '\n',
 <body>
 <div class="col-md-12">
 <div class="row">
 <nav class="navbar navbar-expand-lg navbar-light fixed-top">
 <div class="container-fluid">
 <a class="navbar-brand" href="#">
 <img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
 <span class=

In [11]:
body = list(html.children)[3]
body

<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</span>
</a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-bs-target="#navbarNav" data-bs-toggle="collapse" type="button">
<span class="navbar-toggler-icon"></span>
</button>
<div class="collapse navbar-collapse justify-content-end" id="navbarNav">
<ul class="navbar-nav">
<li class="nav-item">
<a aria-current="page" class="nav-link active" href="#">The Red Planet</a>
</li>
<li class="nav-item">
<a class="nav-link" href="#">The Program</a>
</li>
<li class="nav-item">
<a class="nav-link" href="#">News &amp; Events</a>
</li>
<li class="nav-item">
<a class="nav-link" href="#">Multimedia</a>
</li>
<li class="nav-item">
<a class="na

In [17]:
p = list(body.children)[1]

# for kid in p:
#     p.get_text()

<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</span>
</a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-bs-target="#navbarNav" data-bs-toggle="collapse" type="button">
<span class="navbar-toggler-icon"></span>
</button>
<div class="collapse navbar-collapse justify-content-end" id="navbarNav">
<ul class="navbar-nav">
<li class="nav-item">
<a aria-current="page" class="nav-link active" href="#">The Red Planet</a>
</li>
<li class="nav-item">
<a class="nav-link" href="#">The Program</a>
</li>
<li class="nav-item">
<a class="nav-link" href="#">News &amp; Events</a>
</li>
<li class="nav-item">
<a class="nav-link" href="#">Multimedia</a>
</li>
<li class="nav-item">
<a class="nav-link"

In [18]:
p.get_text()

"\n\n\n\n\nMARS Planet Science\nExploration Program\n\n\n\n\n\n\n\nThe Red Planet\n\n\nThe Program\n\n\nNews & Events\n\n\nMultimedia\n\n\nMissions\n\n\nMore\n\n\n\n\n\n\n\n\n\n\nNews\n\n\n\n\n\n\n\n\nLatest\n\n\n\n\nAll Categories\n\n\n\n\n\n\n\n\n\n\n\n\nMore\n\n\n\n\n\nYou Might Also Like\n\n\n\n\n\n\n\n\n\n\n\nNASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities\n\n\n\n\n\n\n\nThe Launch Is Approaching for NASA's Next Mars Rover, Perseverance\n\n\n\n\n\n\n\nNASA to Hold Mars 2020 Perseverance Rover Launch Briefing\n\n\n\n\n\n\n\n\n\n\n\nWith supporting text below as a natural lead-in to additional content.\n\n\n\n\n\n\n\nWith supporting text below as a natural lead-in to additional content.\n\n\n\n\n\n\n\nWith supporting text below as a natural lead-in to additional content.\n\n\n\n\n\n\n\n\n\n\n\nWith supporting text below as a natural lead-in to additional content.\n\n\n\n\n\n\n\nWith supporting text below as a natural lead-in to additional content.\n\n\n\n\n\n\n

In [19]:
soup.find_all('p')[0].get_text()

'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities'

In [23]:
x = soup.find_all('p')
x

[<p class="card-text">NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities</p>,
 <p class="card-text">The Launch Is Approaching for NASA's Next Mars Rover, Perseverance</p>,
 <p class="card-text">NASA to Hold Mars 2020 Perseverance Rover Launch Briefing</p>,
 <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>,
 <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>,
 <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>,
 <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>,
 <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>,
 <p class="card-text">With supporting text below as a natural lead-in to additional content.</p>]

In [ ]:
# JPL Mars Space Images - Featured Image